In [2]:
import math

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

Вам предлагается предсказать цену квадратного метра недвижимости, используя датасет оценок стоимости квартир

Цель этого курса - всё ещё научить анализировать данные, а не строить ML модели. Поэтому в этой лабораторной мы предлагаем построить новые полезные для предсказания признаки, удалить бесполезные, обработать сырые и может даже поискать дополнительные признаки в интернете

Также разрешается использовать только алгоритм LinearRegression из python пакета sklearn. Использовать другие модели при решении нельзя. Разрешается и рекомендуется придумывать дополнительные признаки для датасета, использовать признаки из сторонних источников и находить наилучшие гиперпараметры алгоритма

Примером признака из сторонних источников могут быть api карт. Сторонние источники вроде сервисов оценки недвижимости использовать нельзя. Преподаватель имеет право забанить такого рода признаки и решение с ними

Баллы ставятся на момент публикации private leaderbord'а, это происходит на следующий день после конца соревнования. Баллы ставятся только в случае, если код решения сдан преподавателю практики

Все студенты получают одинаковое количество баллов за лабораторную, кроме первых и последних трёх.
Первые три места получают +30, 20 и 10 процентов баллов соответственно
Последние три места получают -50, 60, 70 процентов баллов соответственно

In [3]:
#sample_submission_df = pd.read_csv('data\\sample_submission.csv')

#sample_submission_df

In [4]:
df_test = pd.read_csv('data\\test_x.csv')

df_test

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type
0,Москва,1.0,COL_164098,55.700000,37.752365,5,21,51,89,0,...,1048.0,8.510989,8.791667,1970.950549,1971.250000,Москва,4863.0,S5484,2020-05-24,100
1,Октябрьский,NaN,COL_70629,54.489840,53.490696,0,1,7,21,0,...,136.0,4.937500,3.173913,1973.046875,1965.565217,Башкортостан,26.0,S12273,2020-02-23,100
2,Пермь,1.0,COL_132045,57.969273,56.152996,1,21,37,50,0,...,395.0,7.118644,11.071429,1973.639344,1980.400000,Пермский край,242.0,S34019,2020-04-26,100
3,Балашиха,1.0,COL_16510,55.785270,37.853317,0,5,39,49,0,...,NaN,NaN,NaN,NaN,NaN,Московская область,91.9,S21668,2020-01-19,10
4,"Одинцовский городской округ, с. Аксиньино",3.0,COL_141915,55.738186,36.981342,2,8,8,9,0,...,NaN,NaN,NaN,NaN,NaN,Московская область,1500.0,S2158,2020-05-03,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68820,Кемерово,NaN,COL_278008,55.518540,86.096436,1,17,18,19,0,...,578.0,4.816667,5.344828,1979.196721,1983.137931,Кемеровская область,237.7,S25064,2020-08-23,10
68821,Тюмень,NaN,COL_205933,57.153940,65.492440,0,4,30,44,0,...,594.0,6.456522,6.130435,1978.985507,1977.956522,Тюменская область,74.0,S25354,2020-06-28,10
68822,Уфа,7.0,COL_157101,54.729095,55.939934,6,70,128,209,0,...,780.0,5.151316,4.890625,1951.220779,1946.303030,Башкортостан,25.0,S6517,2020-05-24,100
68823,Москва,1.0,COL_130653,55.739864,37.849950,0,47,100,170,0,...,1647.0,13.416667,14.125000,1994.722222,1994.175000,Москва,22.4,S25226,2020-04-19,100


In [5]:
df_train_all = pd.read_csv('data\\train.csv')

df_train_all

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type
0,Ставрополь,NaN,COL_65683,45.011356,41.919273,7,71,144,225,0,...,969.0,8.439655,9.250000,1990.232759,1993.500000,Ставропольский край,31.00,S1842,2020-02-23,10
1,Омск,NaN,COL_159011,54.984833,73.387344,4,40,109,206,0,...,305.0,4.756410,4.333333,1960.346154,1956.291667,Омская область,150.00,S14290,2020-05-24,10
2,Белгород,NaN,COL_205850,50.576805,36.563267,0,9,22,44,0,...,533.0,6.435897,6.400000,1980.452991,1989.633333,Белгородская область,73.00,S25733,2020-06-28,10
3,Большой Исток,NaN,COL_229491,56.714413,60.792454,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,Свердловская область,438.00,S6985,2020-07-19,110
4,Новосибирск,1.0,COL_77283,55.055200,82.895810,0,25,50,101,0,...,1188.0,6.830065,6.672414,1974.045752,1974.637931,Новосибирская область,62.70,S4778,2020-03-01,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206469,Нижний Тагил,1.0,COL_177140,57.911450,59.961727,2,32,56,89,0,...,1031.0,5.466667,5.560000,1966.519380,1967.381818,Свердловская область,41.00,S19208,2020-06-07,100
206470,Новосибирск,NaN,COL_102032,55.067345,82.960770,1,6,8,28,0,...,939.0,6.120370,6.078947,1965.807339,1964.461538,Новосибирская область,98.30,S2332,2020-03-22,110
206471,Ставрополь,NaN,COL_253873,44.998290,41.921482,13,97,157,227,0,...,649.0,8.894737,9.214286,1997.815789,1995.928571,Ставропольский край,921.50,S1743,2020-08-02,100
206472,Санкт-Петербург,5.0,COL_186498,59.855330,30.319840,3,95,274,429,0,...,1224.0,6.386792,8.055556,1961.481132,1962.333333,Санкт-Петербург,40.01,S16774,2020-06-14,10


In [6]:

from math import inf, isnan


def dropUnusableColumns(curr_df):
    out_def_df_numb = curr_df.select_dtypes(exclude=['object'])

    out_def_df_numb['is_negative_floor'] = out_def_df_numb['floor'].apply(lambda x: 1 if (not isnan(x)) and x < 0 else 0)
    out_def_df_numb['is_khrushchyovka'] = out_def_df_numb['floor'].apply(lambda x: 1 if (not isnan(x)) and 0 < x < 10 else 0)
    out_def_df_numb['is_high_floor'] = out_def_df_numb['floor'].apply(lambda x: 1 if (not isnan(x)) and x > 10 else 0)
    out_def_df_numb['is_uknown'] = out_def_df_numb['floor'].apply(lambda x: 1 if isnan(x) else 0)
    out_def_df_numb = out_def_df_numb.drop('floor', axis=1)

    for i in range(len(out_def_df_numb.columns)):
        out_def_df_numb.iloc[:, i]=out_def_df_numb.iloc[:, i].fillna(out_def_df_numb.iloc[:, i].mean())

    return out_def_df_numb

def resultColumnTransformer(curr_df):
    out_def_df_numb = np.log(curr_df)
    return out_def_df_numb

In [7]:
df_test_result = df_train_all.per_square_meter_price
df_test_columns = dropUnusableColumns(df_train_all.drop('per_square_meter_price', axis=1))
df_test_columns

,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,total_square,realty_type,is_negative_floor,is_khrushchyovka,is_high_floor,is_uknown
0,45.011356,41.919273,7,71,144,225,0,0,0,0,...,8.439655,9.250000,1990.232759,1993.500000,31.00,10,0,0,0,1
1,54.984833,73.387344,4,40,109,206,0,1,1,1,...,4.756410,4.333333,1960.346154,1956.291667,150.00,10,0,0,0,1
2,50.576805,36.563267,0,9,22,44,0,1,1,4,...,6.435897,6.400000,1980.452991,1989.633333,73.00,10,0,0,0,1
3,56.714413,60.792454,0,0,0,1,0,0,0,0,...,7.071113,7.383622,1967.588267,1968.017046,438.00,110,0,0,0,1
4,55.055200,82.895810,0,25,50,101,0,0,0,1,...,6.830065,6.672414,1974.045752,1974.637931,62.70,10,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206469,57.911450,59.961727,2,32,56,89,0,0,0,0,...,5.466667,5.560000,1966.519380,1967.381818,41.00,100,0,1,0,0
206470,55.067345,82.960770,1,6,8,28,0,0,0,0,...,6.120370,6.078947,1965.807339,1964.461538,98.30,110,0,0,0,1
206471,44.998290,41.921482,13,97,157,227,0,0,0,0,...,8.894737,9.214286,1997.815789,1995.928571,921.50,100,0,0,0,1
206472,59.855330,30.319840,3,95,274,429,0,2,4,4,...,6.386792,8.055556,1961.481132,1962.333333,40.01,10,0,1,0,0


In [8]:
from sklearn.linear_model import LinearRegression

df_test_formatted_result = resultColumnTransformer(df_test_result)

regressor = LinearRegression()
regressor.fit(df_test_columns, df_test_formatted_result)
df_test_formatted_result

0          9.778217
1         10.409301
2         10.020966
3         11.283627
4         11.751452
            ...    
206469    11.013418
206470    11.222595
206471     9.745967
206472    11.839819
206473    12.048225
Name: per_square_meter_price, Length: 206474, dtype: float64

In [9]:

lab4_res_array = regressor.predict(dropUnusableColumns(df_test))
lab4_res_array

lab4_res_df = pd.DataFrame(lab4_res_array, columns=['per_square_meter_price'])
lab4_res_df['per_square_meter_price'] = np.exp(lab4_res_df['per_square_meter_price'])
lab4_res_df.index.names = ["id"]
lab4_res_df = lab4_res_df.reset_index(level=0)
lab4_res_df = lab4_res_df.reindex(columns=['per_square_meter_price', 'id'])
lab4_res_df.to_csv("res/res_.csv", index=False)
lab4_res_df

,per_square_meter_price,id
0,72158.403402,0
1,23808.120364,1
2,61588.056882,2
3,41526.596875,3
4,43854.026048,4
...,...,...
68820,19599.049276,68820
68821,30590.782826,68821
68822,96217.163336,68822
68823,145228.065765,68823


In [11]:
def correlationCalculate(dataTab, target):
    cor_df = pd.DataFrame(columns=['cor'])
    for col in dataTab.columns:
        cor_df.loc[col] = [target.corr(dataTab[col])]
    return cor_df

correlationCalculate(df_test_columns, df_test_formatted_result).so

,cor
lat,0.156673
lng,-0.275346
osm_amenity_points_in_0.001,0.357936
osm_amenity_points_in_0.005,0.428900
osm_amenity_points_in_0.0075,0.438521
...,...
realty_type,0.168014
is_negative_floor,-0.013997
is_khrushchyovka,0.226472
is_high_floor,0.108561
